In [1]:
from enbios import Experiment
import json
import bw2data

from enbios.bw2.brightway_experiment_adapter import BrightwayAdapter

config = json.load(open("non_linear_config.json", encoding="utf-8"))

In [2]:
exp = Experiment(config)
exp

Excluding 0 filtered results
Excluding 104 filtered results


Experiment: (call info() for details)
Structural nodes: 1
Methods: 1
Hierarchy (depth): 2
Scenarios: 1

In [3]:
bw_adapter_def: dict = config["adapters"][0]
bw_method = bw_adapter_def["methods"]
bw_method

{'GWP1000': ['ReCiPe 2016 v1.03, midpoint (E)',
  'climate change',
  'global warming potential (GWP1000)']}

In [4]:
result = exp.run()
result[Experiment.DEFAULT_SCENARIO_NAME]["results"]

2024-06-05 11:09:08,394 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 11:09:17,569 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 1.570765820650225}}

Let's have a look at the characterization factors. They can be loaded from brightway like this.
Whay we are getting back is a list of tuples. Each tuple has two elements, first a list which identifies the biosphere activity by its database and code, and 2nd the characterization factor for that activity.  `([<database>,<code>], <cf>)`

In [5]:
method_cfs = bw2data.Method(tuple(bw_method["GWP1000"])).load()
# or # bw_adapter.methods["GWP1000"].id
method_cfs

[(['biosphere3', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00'], 1),
 (['biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'], 1),
 (['biosphere3', 'aa7cac3a-3625-41d4-bc54-33e2cf11ec46'], 1),
 (['biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'], 1),
 (['biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'], 1),
 (['biosphere3', 'e1c597cc-14cb-4ebb-af07-7a93a5b77d34'], 1),
 (['biosphere3', '6d89125e-e9b7-4d7e-a1fc-ada45dbd8815'], 1),
 (['biosphere3', '78eb1859-abd9-44c6-9ce3-f3b5b33d619c'], 1),
 (['biosphere3', 'e4e9febc-07c1-403d-8d3a-6707bb4d96e6'], 1),
 (['biosphere3', 'e8787b5e-d927-446d-81a9-f56977bbfeb4'], 1),
 (['biosphere3', '28e1e2d6-97ad-4dfd-932a-9edad36dcab9'], 1),
 (['biosphere3', '4e1f0bb0-2703-4303-bf86-972d810612cf'], 1),
 (['biosphere3', 'd6235194-e4e6-4548-bfa3-ac095131aef4'], 1),
 (['biosphere3', 'eba59fd6-f37e-41dc-9ca3-c7ea22d602c7'], 1),
 (['biosphere3', '73ed05cc-9727-4abf-9516-4b5c0fe54a16'], 1),
 (['biosphere3', '259cf8d6-6ea8-4ccf-84b7-23c930a5b2b3'], -1),
 (['bio

In [6]:
bw_adapter_def["config"]["nonlinear_characterization"]

{'methods': {'GWP1000': {'get_defaults_from_original': True, 'functions': {}}}}

In [7]:
bw_adapter_def["config"]["nonlinear_characterization"]["methods"]["GWP1000"]["get_defaults_from_original"] = True

In [8]:
method_cfs[0][0]

['biosphere3', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00']

In [15]:
bw_adapter_def["config"]["nonlinear_characterization"]["methods"]["GWP1000"]["functions"] = {
    tuple(method_cfs[0][0]): lambda v: v * 2
}

In [16]:
bw_adapter_def["config"]["nonlinear_characterization"]

{'methods': {'GWP1000': {'get_defaults_from_original': True,
   'functions': {('biosphere3',
     'e259263c-d1f1-449f-bb9b-73c6d0a32a00'): <function __main__.<lambda>(v)>}}}}

In [17]:
exp = Experiment(config)
result = exp.run()
result[Experiment.DEFAULT_SCENARIO_NAME]["results"]

Excluding 0 filtered results
Excluding 104 filtered results
2024-06-05 11:10:29,579 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 11:10:36,971 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 1.5707663153907774}}

In [33]:
from typing import cast
from bw2calc import LCA

cast(LCA, exp.get_adapter_by_name("brightway-adapter").lca_objects["default scenario"][0].lca).characterization_matrix

{1169: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method.<locals>.<lambda>(v, cf_=1)>,
 1170: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method.<locals>.<lambda>(v, cf_=1)>,
 1168: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method.<locals>.<lambda>(v, cf_=1)>,
 1171: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method.<locals>.<lambda>(v, cf_=1)>,
 112: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method.<locals>.<lambda>(v, cf_=1)>,
 3635: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method.<locals>.<lambda>(v, cf_=1)>,
 3640: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method.<locals>.<lambda>(v, cf_=1)>,
 3642: <function enbios.bw2.brightway_experiment_adapter.BrightwayAdapter.prepare_nonlinear_method

## Using modules (python files)

Functions can not be included in json files...

### Looking into the inventory and finding a non-zero row

In [16]:
bw_adapter: BrightwayAdapter = exp.get_adapter_by_name("brightway-adapter")
bw_adapter.lca_objects

{'default scenario': [<enbios.bw2.MultiLCA_util.BaseStackedMultiLCA at 0x78e10bce2190>]}

In [17]:
from bw2calc import LCA

enb_lca = bw_adapter.lca_objects["default scenario"][0]
bw_lca: LCA = enb_lca.lca

In [22]:
len(enb_lca.lca.biosphere_matrix.sum(1))

2420

In [20]:
from functools import reduce

reduce(lambda s,x: s + 1 if x != 0.0 else s,enb_lca.lca.biosphere_matrix.sum(1), 0)

2420

In [28]:
from bw2calc.dictionary_manager import ReversibleRemappableDictionary

x: ReversibleRemappableDictionary = enb_lca.lca.dicts.biosphere.original
x._dict

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 13: 11,
 14: 12,
 15: 13,
 17: 14,
 18: 15,
 19: 16,
 20: 17,
 22: 18,
 23: 19,
 27: 20,
 28: 21,
 29: 22,
 33: 23,
 34: 24,
 35: 25,
 36: 26,
 37: 27,
 40: 28,
 41: 29,
 42: 30,
 43: 31,
 44: 32,
 45: 33,
 46: 34,
 47: 35,
 48: 36,
 49: 37,
 53: 38,
 55: 39,
 56: 40,
 57: 41,
 58: 42,
 59: 43,
 62: 44,
 64: 45,
 66: 46,
 67: 47,
 68: 48,
 69: 49,
 70: 50,
 71: 51,
 73: 52,
 74: 53,
 75: 54,
 76: 55,
 78: 56,
 79: 57,
 80: 58,
 81: 59,
 82: 60,
 84: 61,
 85: 62,
 86: 63,
 88: 64,
 91: 65,
 92: 66,
 93: 67,
 94: 68,
 95: 69,
 96: 70,
 97: 71,
 100: 72,
 102: 73,
 104: 74,
 105: 75,
 106: 76,
 107: 77,
 108: 78,
 109: 79,
 110: 80,
 111: 81,
 112: 82,
 113: 83,
 114: 84,
 115: 85,
 116: 86,
 117: 87,
 118: 88,
 126: 89,
 128: 90,
 129: 91,
 130: 92,
 132: 93,
 133: 94,
 135: 95,
 137: 96,
 138: 97,
 139: 98,
 140: 99,
 141: 100,
 142: 101,
 143: 102,
 144: 103,
 146: 104,
 148: 105,
 149: 106,
 151: 107,
 15

In [18]:
bw_lca.inventory.sum(1)

matrix([[3.03474998e-12],
        [7.74894325e-13],
        [7.32632187e-13],
        ...,
        [7.92874439e-06],
        [1.26597255e-07],
        [5.91803546e-06]])

In [40]:
method_cfs[0]

(['biosphere3', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00'], 1)

In [53]:
from bw2data.backends import Activity

carbon_dioxide: Activity = bw2data.Database("biosphere3").get(method_cfs[0][0][1])

In [77]:
new_db = bw2data.Database("temporary")
for activity in new_db:
    print(activity, list(activity.biosphere()))
    for bioflow in activity.biosphere():
        bioflow.delete()
new_db.delete()
new_db.register()
code_name = "new_temp_activity"
activity = new_db.new_activity(code_name, name=code_name, unit="unit")
activity.save()
exc = activity.new_exchange(input=carbon_dioxide, amount=1, type="biosphere")
exc.save()

'new_temp_activity' (unit, GLO, None) []


In [78]:
list(activity.biosphere())

[Exchange: 1 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')) to 'new_temp_activity' (unit, GLO, None)>]

In [79]:
config["hierarchy"]["children"][0]["config"] = {"code": code_name, "default_output": {"unit": "unit", "magnitude": 1}}

In [80]:
exp = Experiment(config)
exp.run()

2024-06-04 15:57:51,332 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-04 15:57:51,480 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'default scenario': {'name': 'root',
  'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 1.0}},
  'output': [{'unit': 'unspecificEcoinventUnit',
    'magnitude': 1.0,
    'label': None}],
  'children': [{'name': 'single_activity',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 1.0}},
    'output': [{'unit': 'unspecificEcoinventUnit',
      'magnitude': 1.0,
      'label': None}],
    'bw_activity_code': 'new_temp_activity'}]}}

In [82]:
exc["amount"] = 2
exc.save()
exp.run()

2024-06-04 15:59:02,789 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-04 15:59:02,908 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'default scenario': {'name': 'root',
  'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 2.0}},
  'output': [{'unit': 'unspecificEcoinventUnit',
    'magnitude': 1.0,
    'label': None}],
  'children': [{'name': 'single_activity',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 2.0}},
    'output': [{'unit': 'unspecificEcoinventUnit',
      'magnitude': 1.0,
      'label': None}],
    'bw_activity_code': 'new_temp_activity'}]}}